In [ ]:
pip install transformers torch faiss-cpu
pip install sentence-transformers
pip install scikit-learn
pip install flask


BERT

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

class QueryUnderstanding:
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained('bert-base-uncased')

    def get_query_embedding(self, query):
        inputs = self.tokenizer(query, return_tensors='pt')
        outputs = self.model(**inputs)
        return outputs.last_hidden_state.mean(dim=1)

query_understanding = QueryUnderstanding()
query_embedding = query_understanding.get_query_embedding("What are the side effects of Aspirin?")
print(query_embedding)


RAG

In [ ]:
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration

class RAGModel:
    def __init__(self):
        self.tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-base")
        self.retriever = RagRetriever.from_pretrained("facebook/rag-token-base", index_name="exact", passages_path="path/to/your/passage_dataset")
        self.model = RagSequenceForGeneration.from_pretrained("facebook/rag-token-base", retriever=self.retriever)

    def generate_answer(self, query):
        input_ids = self.tokenizer(query, return_tensors="pt").input_ids
        generated = self.model.generate(input_ids)
        return self.tokenizer.batch_decode(generated, skip_special_tokens=True)

rag_model = RAGModel()
answer = rag_model.generate_answer("What are the side effects of Aspirin?")
print(answer)


NER

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

class NERModel:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
        self.model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
        self.nlp = pipeline("ner", model=self.model, tokenizer=self.tokenizer)

    def get_entities(self, text):
        return self.nlp(text)

ner_model = NERModel()
entities = ner_model.get_entities("Aspirin may cause stomach upset, heartburn, or nausea.")
print(entities)


Flask

In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

query_understanding = QueryUnderstanding()
rag_model = RAGModel()
ner_model = NERModel()

@app.route('/query', methods=['POST'])
def query():
    data = request.json
    query = data['query']

    # Understand the query
    query_embedding = query_understanding.get_query_embedding(query)

    # Generate the answer
    answer = rag_model.generate_answer(query)

    # Extract entities
    entities = ner_model.get_entities(answer[0])

    return jsonify({
        'query': query,
        'answer': answer[0],
        'entities': entities
    })

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
python app.py
